In [ ]:
#메모리관리
'''
신경망에서는 큰 데이터를 다루는 경우가 많기 때문에 메모리 관리를 제대로 하지 않으면 실행시간이 오래 걸리는 일이 자주 발생.
파이썬의 메모리관리에는 두가지 방법이 있음.
1. 참조 카운트: 참조 수를 세는 방식. 모든 객체는 참조 카운트 0인 상태로 생성되고, 다른 객체가 참조할 때마다 1씩 증가하다가
객체에 대한 참조가 끊길 때마다 1만큼 감소하다가 0이되면 회수한다. 
(순환참조- 참조카운트로는 해결할 수 없는 서로 참조하는 상태이다. a=b=c=none해도 메모리에서 삭제되지 않음.)
2. 가비지 컬렉션(GC): 세대를 기준으로 쓸모없어진 객체를 회수하는 방식. 메모리가 부족해지는 시점에 파이썬 인터프리터에 의해 자동으로 호출된다. 
GC는 순환참조를 해결할 수 있다. 하지만 메모리 해제를 GC에 미루다보면 프로그램의 전체 메모리 사용량이 커지는 원인이 된다.

3. 그래서 나온 것이 weakref모듈(약한 참조) : 다른 객체를 참조하되 참조 카운터는 증가시키지 않는 기능.
    파이썬에서는 weakref.ref 함수를 사용하여 약한 참조를 만든다. b는 약한 참조이고, 약한 참조된 데이터에 접근하려면 b()라고 쓰면 된다.
    a=None을 실행하면 b는 어떻게될까? -> b를 출력하면 dead라는 문작이 나온다.(인스턴스가 삭제됐음을 알 수 있음.)
'''

In [7]:
#약한참조
import weakref
import numpy as np

In [8]:
a = np.array([1, 2, 3])
b = weakref.ref(a)

In [9]:
b

<weakref at 0x0000020190EF5DF0; to 'numpy.ndarray' at 0x000002019145D170>

In [10]:
b()

array([1, 2, 3])

In [12]:
a = None
b            #ndarray 인스턴스가 삭제됐음을 알 수 있다

<weakref at 0x0000020190EF5DF0; to 'numpy.ndarray' at 0x000002019145D170>

In [36]:
#weakref 구조를 DeZero에 도입
import weakref
import numpy as np

In [43]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs] #순환참조를 약한참조로 하겠다는 것
        return outputs if len(outputs) > 1 else outputs[0]


In [44]:
    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [45]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [46]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # outputs를 참조하는 코드 수정. 수정 전엔 gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)


In [47]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [48]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


In [51]:
#DeZero 순환참조 문제 해소. 
'''for문이 두번째 반복될 때 x와 y가 덮어써짐. 사용자는 이전의 계산그래프를 더이상 참조하지않게되고
참조카운트가 0이되므로 이 시점에 계산 그래프에 사용된 메모리가 삭제된다.
'''
for i in range(10):
    x = Variable(np.random.randn(10000)) #큰 데이터
    y = square(square(square(x))) #복잡한 계산을 수행

In [ ]:
#DeZero 메모리 사용을 개선할 수 있는 구조 두가지 도입
'''
1. 역전파 시 사용하는 메모리량을 줄이는 법(불필요한 미분 결과를 보관하지 않고 즉시 삭제)
2. 역전파가 필요 없는 경우의 모드를 제공 (이 모드에서는 불필요한 계산을 생략함)
'''

In [56]:
#DeZero의 역전파 개선
import weakref
import numpy as np

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False): #retain_grad가 true면 모든 변수가 미분 결과를 유지. false면 중간 변수의 미분값을 모두 none으로 재설정
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # 각 함수의 출력 변수의 미분값을 유지하지 않도록 y().grad = none으로 설정(이러면 말단 변수 외에는 미분값 유지x)


In [57]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)

In [59]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()
print(y.grad, t.grad)  #중간변수인 y와 t의 미분값이 삭제되어 그만큼의 메모리를 다른 용도로 사용할 수 있게 됨.
print(x0.grad, x1.grad)

None None
2.0 1.0


In [ ]:
#Function 클래스 복습
'''
DeZero에서 미분-> 순전파 수행 뒤 역전파 수행. 역전파 시에는 순전파의 계산 결과가 필요하기 때문에 순전파 때 결괏값을 기억해야함.
결괏값을 보관하는 로직은 Function 클래스의 self.inputs = inputs 부분.

class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs # <- 이 부분이다.
        self.outputs = [weakref.ref(output) for output in outputs] #순환참조를 약한참조로 하겠다는 것
        return outputs if len(outputs) > 1 else outputs[0]

이와같이 함수는 입력은 inputs라는 인스턴스 변수로 참조한다. 그 결과 inputs가 참조하는 변수의 참조 카운트가 1만큼 증가하고 
call메서드에서 벗어난 뒤에도 메모리에 생존한다. 만약 인스턴스 변수인 inputs로 참조하지 않으면 참조카운트가 0이 되어 메모리에서 삭제된다.
inputs는 역전파 계산시 사용되므로 미리 보관해놔야하지만 미분값이 필요없는 경우가 있다. 이 경우에는 중간계산 결과를 저장할 필요가 없으므로 
계산의 연결도 만들 필요가 없다.
'''

In [60]:
#순전파만을 하는 경우를 만듬. (config클래스 활용)

class Config:
    enable_backprop = True

In [ ]:
#function에서 참조하게 하여 모드를 전환할 수 있게 함
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:  #클래스를 활용한 모드전환. true일 때만 역전파 코드가 실행됨.
            self.generation = max([x.generation for x in inputs])  #세대를 설정
            for output in outputs:
                output.set_creator(self)  #연결설정
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

In [ ]:
#역전파 활성/비활성 모드 전환 - (100,100,100)인 텐서
Config.enable_backprop = True #중간 계산 결과가 계속 유지되어 그만큼 메모리 차지.
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
y.backward()

Config.enable_backprop = False #중간 계산결과는 사용 후 바로 삭제
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))

In [ ]:
#with문을 활용한 모드 전환

#with문을 쓰지 않고 파일에 무언가를 쓰기
f = open('sample.txt', 'w')
f.write('hello world!')
f.close()

#with문 사용
with.open('sample.txt', 'w') as f:
    f.write('hello world!')

#비활성모드
with using_config('enable_backprop', False): #이 안에서만 역전파 비활성 모드가 되고 with블록을 벗어나면 역전파 활성모드로 돌아감
    x = Variable(np.array(2.0))
    y = square(x)

In [64]:
#with문을 사용한 모드 전환 구현 (contextlib 모듈 사용)
import contextlib

@contextlib.contextmanager#문맥을 판단하는 함수가 만들어짐.
def config_test():
    print('start') #전처리
    try:
        yield
    finally:
        print('done') #후처리 / try,finally로 감싸는 이유는 예외가 발생할 수 있기 때문이다.

with config_test():
    print('process...')

start
process...
done


In [ ]:
#위의 내용을 바탕으로 using_config함수 구현
@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

#실제사용
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)

def no_grad():
    return using_config('enable_backprop', False)

#역전파가 필요없는 false일 경우 순전파만 실행. 반복적인 코드를 적기 힘들어서 함수 준비.(no_grad 호출하면 됨)
with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

In [ ]:
#변수 사용성 개선 - Variable 클래스를 덩구 쉽게 사용하도록 개선
'''
변수이름지정 - 변수들을 서로 구분: 많은 변수들을 처리해야하므로 변수에 이름을 붙여줄 수 있도록 설정.
ex)x = Variable(np.array(1.0), 'input_x')라면, 변수 x의 이름은 input_x
'''

class Variable:
    def __init__(self, data, name=None): #인스턴스변수 추가
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name #인스턴스변수
        self.grad = None
        self.creator = None
        self.generation = 0

In [67]:
#ndarray 인스턴스 변수 - 다차원 배열의 형상을 알려줌
import numpy as np
x = np.array([[1,2,3],[4,5,6]])
x.shape

(2, 3)

In [73]:
#shape 메서드 추가 후 실제 데이터의 shape반환. @property 는 shape 메서드를 인스턴스 변수처럼 사용할 수 있게 함.
class Variable:
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape


In [74]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
print(x.shape)

(2, 3)


In [76]:
#len 함수 함께 사용 
'''len을 사용하면 그 안에 포함된 원소의 수를 반환. __len__ 이라는 특수 메서드를 구현하면 Variable 인스턴스에 대해서도 len 함수 사용 가능'''

  def __len__(self):
        return len(self.data)

x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(len(x)) #function에 위 def __len__을 넣고 밑에 print(len(x))로 출력하면 2의 값이 나옴.

IndentationError: unexpected indent (2404732790.py, line 4)

In [77]:
#print 함수를 사용하여 Variable 안의 데이터 내용을 출력
 def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

x = Variable(np.array([[1,2,3],[4,5,6]]))
print(x) # 결괏값은 variable([[1 2 3]
                          #  [4 5 6]]) 이 나옴


IndentationError: unexpected indent (1135604139.py, line 2)

In [78]:
#연산자 오버로드
'''
+와 *를 구현. 곱셈을 수행하는 Mul 클래스를 구현. Variable 인스턴스를 ndarray 인스턴스처럼 사용하도록 구성
(DeZero 를 평범한 넘파이 코드를 작성하듯 사용할 수 있음)

곱셈 연산자 오버로드
곱셈의 특수 메서드는 __mul__(self, other) 이고
__mul__ 메서드를 정의하면 * 연산자를 사용할 때 __mul__ 메서드가 호출된다.
a * b 실행시 호출 순서:  1. 인스턴스 a의 특수 메서드인 __mul__호출
                        2. a -> self, b -> other에 전달됨. (왼왼, 오오)
---------------------------------------------------------------------
스칼라를 출력하는 합성함수를 출력해야하므로 마지막의 L은 오차, 다른말로 손실을 뜻한다. 손실이라는 스칼라를 출력하는 합성함수를 가정하는 것.
'''

'\n+와 *를 구현. 곱셈을 수행하는 Mul 클래스를 구현. Variable 인스턴스를 ndarray 인스턴스처럼 사용하도록 구성\n(DeZero 를 평범한 넘파이 코드를 작성하듯 사용할 수 있음)\n\n곱셈 연산자 오버로드\n곱셈의 특수 메서드는 __mul__(self, other) 이고\n__mul__ 메서드를 정의하면 * 연산자를 사용할 때 __mul__ 메서드가 호출된다.\na * b 실행시 호출 순서:  1. 인스턴스 a의 특수 메서드인 __mul__호출\n                        2. a -> self, b -> other에 전달됨. (왼왼, 오오)\n---------------------------------------------------------------------\n스칼라를 출력하는 합성함수를 출력해야하므로 마지막의 L은 오차, 다른말로 손실을 뜻한다. 손실이라는 스칼라를 출력하는 합성함수를 가정하는 것.\n'

In [79]:
#MUL 클래스의 코드
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


In [80]:
#MUL클래스를 파이썬 함수로 사용할 수 있도록 해줌 
def mul(x0, x1):
    return Mul()(x0, x1)

In [84]:
#MUL함수를 이용한 곱셈

import weakref
import numpy as np
import contextlib


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a, b), c)
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


In [85]:
# y = a*b+c의 형태로 확장 (연산자 오버로드를 사용)
import weakref
import numpy as np
import contextlib


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    '''
    def __mul__(self, other):
        return mul(self, other) 
    '''

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


def mul(x0, x1):
    return Mul()(x0, x1) #mul클래스를 파이썬 함수로 사용할 수 있도록 함


Variable.__add__ = add
Variable.__mul__ = mul #특수메서드 __mul__

a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = a * b + c
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


In [ ]:
#ndarray와 함께 사용하기
'''
a가 variable 인스턴스일 때 a*np.array(2.0)코드를 만날때 ndarray 인스턴스를 자동으로 Variable 인스턴스로 변환함
as_variable 함수 구현 - 인수로 주어진 객체를 Variable 인스턴스로 변환해주는 함수
* inputs = [as_variable(x) for x in inputs] -> x = Variable(np.array(2.0)) 
                                                y = x + np.array(3.0)
                                                print(y)
'''

#float, int와 함께 사용하기
'''
파이썬의 자료형과 사용 가능하도록 개선 - 파이썬의 float와 int, np.float64와 np.int64 같은 타입과도 함께 사용 하도록 
x가 Variable 인스턴스일 때 x + 3.0 코드를 실행

'''
#as_array 함수 사용
'''
as_array 를 사용하면 x1이 float나 int 인 경우 ndarray 인스턴스로 변환됨. 
ndarray 인스턴스는 Function 클래스에서 Variable 인스턴스로 변환됨.

'''
#하지만 지금의 방식에는 두가지 문제가 남아있음.

In [ ]:
#문제점1. 첫번째 인수가 float나 int인 경우.
'''
y = 2.0*x 코드수행 -> TypeError 오류가 발생
오류발생과정: 1. 연산자 왼쪽에 __mul__메서드 호출시도
            2. 2.0은 float 타입이므로 __mul__ 메서드는 구현되어있지 x
            3. * 연산자 오른쪽에 있는 x의 특수 메서드를 호출 시도
            4. x가 오른쪽에 있기 때문에 __rmul__ 메서드를 호출 시도
            5. Variable 인스턴스에는 __rmul__ 메서드가 구현되어 있지 않음.
해결방법-> __rmul__ 메서드를 구현하면 해결
'''
#문제점2. 좌항이 ndarray 인스턴스인 경우
'''
x = Variable(np.array([1.0]))
y = np.array([2.0]) + x 의 경우 좌항은 ndarray 인스턴스이고 우항은 Variable 인스턴스이다.
좌항인 ndarray 인스턴스의 __add__메서드가 호출. 하지만 내가 원하는건 우항인 Variable 인스턴스의 __radd__메서드가 호출되길 원함.

해결방법-> 연산자 우선순위를 지정. 구체적으로는 Variable 인스턴스의 속성에 __arrayy_priority__를 추가하고 그 값을 큰 정수로 설정해야함.
코드: Class Variable:
        __array_priority__ = 200 
        
        이렇게하면 Variable 인스턴스의 연산자 우선순위를 ndarray 인스턴스의 연산자 우선순위보다 높일 수 있다.
        좌항이 ndarray 인스턴스라 해도 Variable 인스턴스의 연산자 메서드가 우선적으로 호출되는 것이다.

'''

In [ ]:
#연산자 추가. (/, - 등)
'''
연산자 추가 순서 : 1. Function 클래스를 상속하여 원하는 함수 클래스를 구현
                2. 파이썬 함수로 사용할 수 있도록 함
                3. Variable 클래스의 연산자를 오버로드함
'''

In [ ]:
#음수(부호변환)에서 추가할 부분

class Neg(Function):
    def forward(self, x):
        return -x

    def backward(self, gy):
        return -gy


def neg(x):
    return Neg()(x)

Variable.__neg__ = neg

-----------------

x = Variable(np.array(2.0))
y = -x #부호를 바꾼다.
print(y) # 결과는 variable(-2.0)

In [ ]:
#뺄셈

class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy


def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)


Variable.__sub__ = sub

---------------------------------

#우항인 경우 추가

def rsub(x0, x1):
    x1 = as_array(x1)
    return sub(x1, x0)

Variable.__rsub__ = rsub 

--------------------------------
x = Variable(np.array(2.0))
y1 = 2.0 - x
y2 = x - 1.0
print(y1)
print(y2) # 결과 : variable(0.0)
                 #variable(1.0)


In [ ]:
#나눗셈

class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1

def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)

def rdiv(x0, x1):
    x1 = as_array(x1)
    return div(x1, x0) #

Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv

In [ ]:
#거듭제곱

class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c

        gx = c * x ** (c - 1) * gy
        return gx

def pow(x, c):
    return Pow(c)(x)

Variable.__pow__ = pow

-----------------------------------
x = Variable(np.array(2.0))
y = x **3
print(y) #결과: variable(8.0)

In [ ]:
#지금까지의 성과를 재사용할 수 있는 패키지로 정리
'''
*모듈: 파이썬 파일. 특히 다른 파이썬 프로그램에서 임포트 하여 사용하는 것을 가정하고 만들어진 파이썬 파일.

*패키지: 여러 모듈을 묶은 것. 패키지를 만들려면 먼저 디렉터리를 만들고 그 안에 모듈을 추가한다.

*라이브러리: 여러 패키지를 묶은 것. 하나이상의 디렉터리로 구성되고 때로는 패키지를 가리켜 라이브러리라고 부르기도 한다. 

dezero라는 공통의 디렉터리를 하나 만들고 모듈을 추가한다. 그리하여 dezero라는 패키지가 만들어지는데 
이것이 바로 우리가 만드는 프레임워크이다.

<파일구성>
dezero 패키지 – 딥 러닝 프레임워크
steps 디렉토리 – step01.py ~ step60.py 

<코어 클래스로 옮기기>
step22.py 코드를 dezero/core_simple.py 코어 파일로 옮김(지금까지 구현한 기능들이 DeZero의 핵심이기 때문)

import numpy as np
from dezero.core_simple import Variable

x = Variable(np.array(1.0))
print(y) 코드가 정상 동작 되어야 한다.

<연산자 오버로드>
오버로드한 연산자들을 dezero로 옮김
코어 파일인 dezero/core_simple.py에 다음 setup_variable 함수들을 추가
dezero/__init__.py 파일에서 호출
dezero/__init__.py 는 모듈을 임포트할 때 가장 먼저 실행 되는 파일
이렇게 함으로써 dezero 패키지를 이용하는 사용자는 반드시 연산자 오버로드가 이루어진 상태에서 Variable을 사용할 수 있음

<실제 _init_.py 파일>
is_simple_core 플래그로 임포트 대상을 선택(True - core_simple.py, False - core.py에서 임포트 이루어짐)

<dezero 임포트 하기>
dezero라는 패키지가 만들어지고 
step23.py는 아래의 코드가 동작되어야 한다.
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))

import numpy as np
from dezero import Variable

x = Variable(np.array(1.0))
y = (x+3) ** 2
y.backward()

print(y)
print(x.grad)

'''